#Lasso and ridge regression


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/content/drive/MyDrive/ML/housing_FULL.csv')
display(data)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0


In [4]:
columns = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount',
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price'
]
data = data[columns]
data.head()


,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


In [5]:
data.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,3
Distance,1
CouncilArea,3
Bedroom2,8217


In [6]:
#columns to fill null with 0
columns_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
data[columns_to_fill_zero] = data[columns_to_fill_zero].fillna(0)
data.isna().sum()


,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,0
Distance,0
CouncilArea,3
Bedroom2,0


In [7]:
#columns to fill null with mean
data['Landsize'] = data['Landsize'].fillna(data['Landsize'].mean())
data['BuildingArea'] = data['BuildingArea'].fillna(data['BuildingArea'].mean())
data.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,0
Distance,0
CouncilArea,3
Bedroom2,0


In [8]:
#Now drop rows with null
data.dropna(inplace=True)
data.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,0
Propertycount,0
Distance,0
CouncilArea,0
Bedroom2,0


In [9]:
x = data.drop('Price', axis=1)
y = data['Price']

In [10]:
#one hot encoding non numeric features
x = pd.get_dummies(x, columns=['Suburb', 'Type', 'Method', 'SellerG', 'Regionname', 'CouncilArea'])
x.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Suburb_Abbotsford,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,4019.0,2.5,2.0,1.0,1.0,202.0,160.2564,True,False,...,False,False,False,False,False,False,False,False,True,False
2,2,4019.0,2.5,2.0,1.0,0.0,156.0,79.0000,True,False,...,False,False,False,False,False,False,False,False,True,False
4,3,4019.0,2.5,3.0,2.0,0.0,134.0,150.0000,True,False,...,False,False,False,False,False,False,False,False,True,False
5,3,4019.0,2.5,3.0,2.0,1.0,94.0,160.2564,True,False,...,False,False,False,False,False,False,False,False,True,False
6,4,4019.0,2.5,3.0,1.0,2.0,120.0,142.0000,True,False,...,False,False,False,False,False,False,False,False,True,False


In [11]:
#split the data in training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)

In [12]:
x_train.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Suburb_Abbotsford,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
12799,2,3593.0,4.4,2.0,1.0,1.0,0.000000,62.0000,False,False,...,False,False,False,False,False,False,False,False,False,False
477,3,4836.0,6.3,0.0,0.0,0.0,593.598993,160.2564,False,False,...,False,False,False,False,True,False,False,False,False,False
29182,3,16166.0,14.7,3.0,2.0,2.0,580.000000,160.2564,False,False,...,False,False,False,False,False,False,False,True,False,False
21339,3,8801.0,8.4,3.0,1.0,2.0,593.598993,160.2564,False,False,...,False,False,False,False,True,False,False,False,False,False
22275,3,11918.0,5.2,3.0,1.0,2.0,212.000000,160.2564,False,False,...,False,True,False,False,False,False,False,False,False,False


#Linear regression

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

regressor = LinearRegression()
regressor.fit(x_train, y_train)

print('For training dataset')
print('R2 score :',r2_score(y_train,regressor.predict(x_train)))
print('MSE :',mean_squared_error(y_train,regressor.predict(x_train)))

print('\nFor testing dataset')
print('R2 score :',r2_score(y_test,regressor.predict(x_test)))
print('MSE :',mean_squared_error(y_test,regressor.predict(x_test)))

For training dataset
R2 score : 0.6827792395792723
MSE : 132454004507.99455

For testing dataset
R2 score : 0.13871990247613764
MSE : 342202734617.9804


#L1 regularization

In [14]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=50)
lasso.fit(x_train, y_train)

print('For training dataset')
print('R2 score :',r2_score(y_train,lasso.predict(x_train)))
print('MSE :',mean_squared_error(y_train,lasso.predict(x_train)))

print('\nFor testing dataset')
print('R2 score :',r2_score(y_test,lasso.predict(x_test)))
print('MSE :',mean_squared_error(y_test,lasso.predict(x_test)))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.698e+14, tolerance: 7.963e+11
  model = cd_fast.enet_coordinate_descent(


For training dataset
R2 score : 0.6767441886056493
MSE : 134973911048.17824

For testing dataset
R2 score : 0.6636167960513981
MSE : 133651355234.73553


#L2 regularization

In [15]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=50)
ridge.fit(x_train, y_train)

print('For training dataset')
print('R2 score :',r2_score(y_train,ridge.predict(x_train)))
print('MSE :',mean_squared_error(y_train,ridge.predict(x_train)))

print('\nFor testing dataset')
print('R2 score :',r2_score(y_test,ridge.predict(x_test)))
print('MSE :',mean_squared_error(y_test,ridge.predict(x_test)))

For training dataset
R2 score : 0.6627818359147762
MSE : 140803824335.71436

For testing dataset
R2 score : 0.6673612969137435
MSE : 132163594820.25728


#K cross validation

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
from sklearn.model_selection import cross_val_score

lasso_r2 = cross_val_score(lasso, x, y, cv=5, scoring='r2')
lasso_mse = -cross_val_score(lasso, x, y, cv=5, scoring='neg_mean_squared_error')

ridge_r2 = cross_val_score(ridge, x, y, cv=5, scoring='r2')
ridge_mse = -cross_val_score(ridge, x, y, cv=5, scoring='neg_mean_squared_error')

print("Lasso cross validation  mean R2 Score :", lasso_r2.mean())
print("Lasso cross validation  mean MSE :", lasso_mse.mean())
print("\nRidge cross validation  mean R2 Score :", ridge_r2.mean())
print("Ridge cross validation  mean MSE :", ridge_mse.mean())

Lasso cross validation  mean R2 Score : 0.6292208728546149
Lasso cross validation  mean MSE : 151056539057.89673

Ridge cross validation  mean R2 Score : 0.6218213392776362
Ridge cross validation  mean MSE : 153891731638.3587
